### create NETCDF data files to publish with paper

In [ ]:
### create netcdf of full winter season (01/10 - 30/04) for seasons where data is available, for each satellite

import numpy as np
import datetime
import os
import glob
import netCDF4 as nc4
import xarray as xr

# load auxiliary
lats = np.load('/home/cjn/OI_PolarSnow/EASE/auxiliary/new_lat_25km.npy')
lons = np.load('/home/cjn/OI_PolarSnow/EASE/auxiliary/new_lon_25km.npy')
datapath = '/home/cjn/OI_PolarSnow/EASE/freeboard_daily_interpolated/25km/daily_numpys/'

seasons = ['2010-2011','2011-2012','2012-2013','2013-2014','2014-2015','2015-2016','2016-2017','2017-2018',
           '2018-2019','2019-2020']
sats = ['CS2_CPOM', 'CS2_LARM', 'CS2S3_CPOM', 'CS2S3_LARM', 'AK_CPOM', 'AK_LARM', 'IS2']

for sat in sats:
    location = datapath + sat + '/'
    print(sat,' starting')
    for season in seasons:
        if season in ['2011-2012','2015-2016','2019-2020']:
            days = np.arange(0,213)
        else:
            days = np.arange(0,212)
        dates = [datetime.date(int(season[:4]),10,1)+datetime.timedelta(days=int(days)) for days in days]

        fb = np.full((len(days),360,360), np.nan)
        unc = np.full((len(days),360,360), np.nan)
        files = sorted(glob.glob(location+'*'+season+'*.npy'))

        if len(files)>0:
            for day in days:
                date = dates[day].strftime('%Y%m%d')
                for f in os.listdir(location):
                    if 'FB_interp' in f:
                        if date in f:
                            fb[day] = np.load(location+str(f))
                        else:
                            pass
                    if 'FB_uncertainty' in f:
                        if date in f:
                            unc[day] = np.load(location+str(f))
                        else:
                            pass

            # create NETCDF
            if sat == 'IS2':
                data_vars = {'Laser Freeboard':(['t','x','y'],fb),
                            'Uncertainty':(['t','x','y'],unc)}
                ds = xr.Dataset(data_vars = data_vars,
                                coords={'Longitude':(['x','y'],lons),
                                        'Latitude':(['x','y'],lats),
                                         'Day':(['t'],days)}     )
                ds.attrs = {'Description': "daily-resolution interpolated laser freeboard data on 25km EASE-Grid 2.0 (Nab et al. 2022)",
                            'Laser Freeboard': "daily-resolution laser freeboard (m)",
                            'Uncertainty': "daily-resolution laser freeboard uncertainty (m)",
                            'Day': "day of season since 1 October"}
                ds.to_netcdf(f'/home/cjn/OI_PolarSnow/EASE/freeboard_daily_interpolated/25km/NETCDFs/'+sat+'_'+season+'.nc','w')
            else:

                data_vars = {'Radar Freeboard':(['t','x','y'],fb),
                            'Uncertainty':(['t','x','y'],unc)}
                ds = xr.Dataset(data_vars = data_vars,
                                coords={'Longitude':(['x','y'],lons),
                                        'Latitude':(['x','y'],lats),
                                         'Day':(['t'],days)}     )
                ds.attrs = {'Description': "daily-resolution interpolated radar freeboard data on 25km EASE-Grid 2.0 (Nab et al. 2022)",
                            'Radar Freeboard': "daily-resolution radar freeboard (m)",
                            'Uncertainty': "daily-resolution radar freeboard uncertainty (m)",
                            'Day': "day of season since 1 October"}
                ds.to_netcdf(f'/home/cjn/OI_PolarSnow/EASE/freeboard_daily_interpolated/25km/NETCDFs/'+sat+'_'+season+'.nc','w')

        else: pass            